# Препроцессинг данных и простая нейронная сеть (Keras) для распознавания рукописных букв

In [2]:
import pandas as pd
import tensorflow as tf
import numpy as np

In [3]:
path = "data/letter-recognition.data"
data = pd.read_csv(path, header= None)

In [3]:
data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,T,2,8,3,5,1,8,13,0,6,6,10,8,0,8,0,8
1,I,5,12,3,7,2,10,5,5,4,13,3,9,2,8,4,10
2,D,4,11,6,8,6,10,6,2,6,10,3,7,3,7,3,9
3,N,7,11,6,6,3,5,9,4,6,4,4,10,6,10,2,8
4,G,2,1,3,1,1,8,6,6,6,6,5,9,1,7,5,10


In [4]:
data.shape

(20000, 17)

In [5]:
labels = data[0]
input_data = data.drop(0, axis=1)

In [6]:
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.losses import *
from keras.metrics import *
from keras.callbacks import *

Using TensorFlow backend.


In [7]:
data.describe()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
count,20000.000000,20000.000000,20000.000000,20000.00000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.00000,20000.000000,20000.000000,20000.000000,20000.000000,20000.00000
mean,4.023550,7.035500,5.121850,5.37245,3.505850,6.897600,7.500450,4.628600,5.178650,8.282050,6.45400,7.929000,3.046100,8.338850,3.691750,7.80120
std,1.913212,3.304555,2.014573,2.26139,2.190458,2.026035,2.325354,2.699968,2.380823,2.488475,2.63107,2.080619,2.332541,1.546722,2.567073,1.61747
min,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000
25%,3.000000,5.000000,4.000000,4.00000,2.000000,6.000000,6.000000,3.000000,4.000000,7.000000,5.00000,7.000000,1.000000,8.000000,2.000000,7.00000
50%,4.000000,7.000000,5.000000,6.00000,3.000000,7.000000,7.000000,4.000000,5.000000,8.000000,6.00000,8.000000,3.000000,8.000000,3.000000,8.00000
75%,5.000000,9.000000,6.000000,7.00000,5.000000,8.000000,9.000000,6.000000,7.000000,10.000000,8.00000,9.000000,4.000000,9.000000,5.000000,9.00000
max,15.000000,15.000000,15.000000,15.00000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.00000,15.000000,15.000000,15.000000,15.000000,15.00000


In [8]:
labels.unique()

array(['T', 'I', 'D', 'N', 'G', 'S', 'B', 'A', 'J', 'M', 'X', 'O', 'R',
       'F', 'C', 'H', 'W', 'L', 'P', 'E', 'V', 'Y', 'Q', 'U', 'K', 'Z'],
      dtype=object)

In [9]:
data.groupby(0).size()

0
A    789
B    766
C    736
D    805
E    768
F    775
G    773
H    734
I    755
J    747
K    739
L    761
M    792
N    783
O    753
P    803
Q    783
R    758
S    748
T    796
U    813
V    764
W    752
X    787
Y    786
Z    734
dtype: int64

In [10]:
#преобразование данных
from sklearn.preprocessing import LabelEncoder, normalize
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

labels_num = LabelEncoder().fit_transform(labels)
labels_output = to_categorical(labels_num)
labels_output = normalize(labels_output)
input_train, input_test, labels_out_train, labels_out_test = train_test_split(input_data, labels_output, test_size=0.3)

In [39]:
n_input = input_data.shape[1]
n1 = 256
n2 = 128
n_output = len(labels.unique())

In [40]:
model = Sequential()  #один способ задания NN
model.add(BatchNormalization(input_shape=(n_input,)))
model.add(Dense(n1, input_shape=(n_input,)))
model.add(Activation('elu'))
model.add(Dropout(rate=0.4))
model.add(Dense(n1))
model.add(Activation('relu'))
model.add(Dropout(rate=0.2))
model.add(Dense(n1))
model.add(Activation('elu'))
# model.add(Dropout(rate=0.1))
model.add(Dense(n1))
model.add(Activation('relu'))
model.add(Dense(n_output))
model.add(Softmax())

In [42]:
model.compile(RMSprop(lr=0.0001), loss = 'categorical_crossentropy', metrics = ['categorical_accuracy'])

filepath = "letter_recognition_checkpoints/weights-{val_categorical_accuracy:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, save_best_only=True, monitor='val_categorical_accuracy', mode='max')
#сохрани checkpoint, если он лучше всех предыдущих по заданной метрике


In [55]:
model.fit(input_train, labels_out_train, batch_size=200, verbose=2, epochs=500, # verbose - говорливая тренировка
          validation_data=(input_test, labels_out_test), callbacks = [checkpoint]) 



Train on 14000 samples, validate on 6000 samples
Epoch 1/10
 - 1s - loss: 0.0351 - categorical_accuracy: 0.9884 - val_loss: 0.0677 - val_categorical_accuracy: 0.9817
Epoch 2/10
 - 1s - loss: 0.0322 - categorical_accuracy: 0.9881 - val_loss: 0.0671 - val_categorical_accuracy: 0.9815
Epoch 3/10
 - 1s - loss: 0.0325 - categorical_accuracy: 0.9886 - val_loss: 0.0679 - val_categorical_accuracy: 0.9822
Epoch 4/10
 - 1s - loss: 0.0389 - categorical_accuracy: 0.9863 - val_loss: 0.0707 - val_categorical_accuracy: 0.9805
Epoch 5/10
 - 1s - loss: 0.0304 - categorical_accuracy: 0.9893 - val_loss: 0.0687 - val_categorical_accuracy: 0.9808
Epoch 6/10
 - 1s - loss: 0.0337 - categorical_accuracy: 0.9881 - val_loss: 0.0655 - val_categorical_accuracy: 0.9807
Epoch 7/10
 - 1s - loss: 0.0332 - categorical_accuracy: 0.9886 - val_loss: 0.0670 - val_categorical_accuracy: 0.9823
Epoch 8/10
 - 1s - loss: 0.0345 - categorical_accuracy: 0.9881 - val_loss: 0.0637 - val_categorical_accuracy: 0.9817
Epoch 9/10
 - 1

In [56]:
model.evaluate(input_test, labels_out_test)

6000/6000 [==============================] - 1s 85us/step


[0.06696850830529971, 0.9816666666666667]